
# 🟦 05 — Agents Hands-On in Node.js (Option A)

This notebook delivers a **Node.js Agent Engineering Starter Kit**, mirroring the Python version from Notebook 04.

It includes:

- Minimal ReAct-style agent loop in JavaScript  
- Tool system (tool registry, execution model)  
- Mock LLM interface (pluggable for real OpenAI / Groq / local models)  
- Memory-enabled agent  
- Multi-agent (Manager → Worker) pattern  
- Hooks for later RAG + MCP integration  

This is the *reference code foundation* for building JS-based agents.


## 1. Minimal Project Setup

In [ ]:
// This is demonstration code — copy into app.js or agents.js in a Node project.

// Placeholder LLM completion — replace with real OpenAI/Groq calls.
async function llmComplete(systemPrompt, messages) {
  return `THOUGHT: I will call the echo tool.
ACTION: echo
ACTION_INPUT: {"text": "Hello from mock Node LLM"}`;
}


## 2. Tool Abstraction

In [ ]:
class Tool {
  constructor(name, description, func) {
    this.name = name;
    this.description = description;
    this.func = func;
  }

  async call(input) {
    return await this.func(input);
  }
}

class ToolRegistry {
  constructor() {
    this.tools = new Map();
  }

  register(tool) {
    if (this.tools.has(tool.name)) {
      throw new Error(`Tool '${tool.name}' already exists.`);
    }
    this.tools.set(tool.name, tool);
  }

  get(name) {
    return this.tools.get(name);
  }

  list() {
    return Array.from(this.tools.values());
  }
}


## 3. Example Tools

In [ ]:
const NOTES = [
  { id: 1, content: "Node agents can use tools and memory." },
  { id: 2, content: "RAG combines retrieval with generation." },
  { id: 3, content: "MCP provides a capability layer via tools." },
];

async function echoTool(input) {
  return { text: input.text || "" };
}

async function addNumbersTool(input) {
  const a = Number(input.a ?? 0);
  const b = Number(input.b ?? 0);
  return { result: a + b };
}

async function searchNotesTool(input) {
  const q = (input.query || "").toLowerCase();
  const matches = NOTES.filter(n => n.content.toLowerCase().includes(q));
  return { matches };
}

const registry = new ToolRegistry();
registry.register(new Tool("echo", "Echo text", echoTool));
registry.register(new Tool("add_numbers", "Add two numbers", addNumbersTool));
registry.register(new Tool("search_notes", "Search notes", searchNotesTool));


## 4. ReAct Parser

In [ ]:
function parseReActResponse(text) {
  const thought = /THOUGHT:(.*)/.exec(text)?.[1]?.trim() || "";
  const action = /ACTION:(.*)/.exec(text)?.[1]?.trim() || "NONE";
  const rawInput = /ACTION_INPUT:(.*)/s.exec(text)?.[1]?.trim() || "{}";

  let actionInput = {};
  try {
    actionInput = JSON.parse(rawInput);
  } catch {
    actionInput = { raw: rawInput, parse_error: true };
  }
  return { thought, action, actionInput };
}


## 5. ReAct Agent Implementation in Node

In [ ]:
class SimpleAgent {
  constructor(tools, maxSteps = 5) {
    this.tools = tools;
    this.maxSteps = maxSteps;
    this.memory = [];
  }

  async run(userQuery) {
    this.memory = [{ user: userQuery }];

    for (let i = 0; i < this.maxSteps; i++) {
      const messages = this.buildMessages(userQuery);
      messages.push({
        role: "user",
        content: `User question: ${userQuery}
Tools:
${this.tools.list().map(t => `- ${t.name}: ${t.description}`).join("\n")}`
      });

      const raw = await llmComplete("system prompt placeholder", messages);
      const { thought, action, actionInput } = parseReActResponse(raw);

      const step = { thought, action, actionInput };

      if (action.toUpperCase() === "NONE") {
        this.memory.push(step);
        return thought;
      }

      const tool = this.tools.get(action);
      if (!tool) {
        step.observation = { error: `Unknown tool '${action}'` };
        this.memory.push(step);
        return `Error: Unknown tool '${action}'`;
      }

      const obs = await tool.call(actionInput);
      step.observation = obs;
      this.memory.push(step);
    }

    return "Max steps reached without completion.";
  }

  buildMessages() {
    const messages = [];
    for (const s of this.memory) {
      if (s.user) messages.push({ role: "user", content: s.user });
      if (s.thought) messages.push({ role: "assistant", content: `THOUGHT: ${s.thought}` });
      if (s.action) messages.push({ role: "assistant", content: `ACTION: ${s.action}` });
      if (s.observation) messages.push({ role: "user", content: `OBS: ${JSON.stringify(s.observation)}` });
    }
    return messages;
  }
}


### Test SimpleAgent

In [ ]:
(async () => {
  const agent = new SimpleAgent(registry, 3);
  const res = await agent.run("Say hi using the echo tool.");
  console.log(res);
})();


## 6. Memory-Aware Agent

In [ ]:
class MemoryAwareAgent extends SimpleAgent {
  summarize() {
    return this.memory.map((s, i) => JSON.stringify(s)).join("\n");
  }

  buildMessages() {
    const summary = this.summarize();
    return [{ role: "system", content: `Memory:
${summary}` }];
  }
}

(async () => {
  const memAgent = new MemoryAwareAgent(registry, 3);
  const ans = await memAgent.run("Search notes about RAG.");
  console.log(ans);
})();


## 7. Multi-Agent Pattern (Manager → Worker)

In [ ]:
class WorkerAgent extends SimpleAgent {
  async runTask(task) {
    return await this.run(task);
  }
}

class ManagerAgent extends SimpleAgent {
  constructor(tools, worker, maxSteps = 3) {
    super(tools, maxSteps);
    this.worker = worker;
  }

  async run(userQuery) {
    const subtask = `Do the main work for: ${userQuery}`;
    const workerResult = await this.worker.runTask(subtask);
    return `Manager delegated. Worker result: ${workerResult}`;
  }
}

(async () => {
  const worker = new WorkerAgent(registry, 3);
  const manager = new ManagerAgent(registry, worker, 2);
  const answer = await manager.run("Explain how agents use tools.");
  console.log(answer);
})();



## 8. How This Extends to RAG + MCP

Later in this universe you will:

- Replace `search_notes` with **real vector DB retrieval**
- Add **RAG planning + reflection loops**
- Replace the simple Tool system with **MCP tool interfaces**
- Build multi-agent RAG pipelines in Node  
- Integrate LangChain.js or custom pipelines  

This notebook forms the *coding foundation* for all future JavaScript agent notebooks.
